# Data Generation
## Generating cross-sell & upsell data with SVD
- Select customers of certain profile - high value (premium/income) / dependences/ recency, and use rules to set them to purchase additional policies
- To maintain consistent customer profile, after synthesizing the data, we would substitute the customer profile and size with that of data before.
    1. GaussianCopulaSynthesizer
    2. Constraint
    3. Conditional Generation
- Aim to get like 20% of the customers with either upsell/xsell

## Incorperating business insights/trends
Profile to synthesis (for Xsell/Upsell)

1. Income/Proxy for alfluence

    - High-income earners (> $90,000) are more likely to have larger policies with broader coverage, such as Investment and Whole Life policies.
    - Mid-income earners ($60,000 - $90,000) often choose Critical Illness or Medical policies.
    - Affluence customer would seek more coverage.

2. Marital Status and Dependents:

    - Married customers with dependents tend to opt for policies with higher coverage, ensuring family protection.
    - Single individuals often select policies like Term Life for basic coverage.

3. Health Conditions:

    - Customers with existing health conditions (e.g., Diabetes, High Blood Pressure) often choose Critical Illness or Medical policies to cover potential medical expenses.

4. Under insured

    - Customers with high premium (Due to investment/endowment) yet little to no insured amount can be considered as a prospect.

### Not Used

4. Payment History:

Customers with consistent on-time payments might receive discounts or premium breaks, indicating reliability.
Those with late payments could be flagged for potential churn and might benefit from personalized payment plans or reminders.



5. Policy Status and Future Opportunities:

Customers nearing the end of their Term Life policy could be approached with options to convert to Whole Life policies.
High-value policyholders with increasing incomes might be suitable candidates for upgrading their insured amount.

## Library

In [123]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from dateutil.relativedelta import relativedelta

import math

# Input

In [124]:
data = pd.read_parquet("data/L0/mock_rules_enriched_data.parquet")

In [125]:
# Creating Flag for Conditions
data["under_insured"] = (data["Income"] > 40000) & (data["Insured_Amount"] == 0)
data["high_income"] = data["Income"] > 70000
data["dependents_needs"] = data["Dependents"] > 2
data["recent_purchase"] = data["Recent_Purchase_Months"] < 1

# SDV
### Metadata

In [126]:
from sdv.metadata import SingleTableMetadata

metadata = SingleTableMetadata()
metadata.detect_from_dataframe(data)

In [127]:
metadata.to_dict()

{'primary_key': 'CustomerID',
 'METADATA_SPEC_VERSION': 'SINGLE_TABLE_V1',
 'columns': {'CustomerID': {'sdtype': 'id'},
  'Age': {'sdtype': 'numerical'},
  'Gender': {'sdtype': 'categorical'},
  'Income': {'sdtype': 'numerical'},
  'Marital_Status': {'sdtype': 'categorical'},
  'Dependents': {'sdtype': 'categorical'},
  'Claims': {'sdtype': 'categorical'},
  'Policy': {'sdtype': 'categorical'},
  'Premium_Amount': {'sdtype': 'numerical'},
  'Insured_Amount': {'sdtype': 'numerical'},
  'Payment_Timeliness': {'sdtype': 'categorical'},
  'Payment_Frequency': {'sdtype': 'categorical'},
  'Payment_Method': {'sdtype': 'categorical'},
  'Recent_Payment_Months': {'sdtype': 'numerical'},
  'Recent_Purchase_Months': {'sdtype': 'numerical'},
  'Policy_Purchase_Years': {'sdtype': 'numerical'},
  'Customer_Tenure_Years': {'sdtype': 'numerical'},
  'Smoker': {'sdtype': 'categorical'},
  'Policy_Issue_Date': {'sdtype': 'datetime', 'datetime_format': '%Y-%m-%d'},
  'Policy_Terminate_Date': {'sdtype': 

In [128]:
from sdv.single_table import GaussianCopulaSynthesizer

synthesizer = GaussianCopulaSynthesizer(metadata)

/home/howt/mambaforge/envs/py312syndata/lib/python3.12/site-packages/sdv/single_table/base.py:92: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


### Constraint

In [129]:
# Positive
recent_purchase_months_constraint = {
    "constraint_class": "Positive",
    "constraint_parameters": {
        "column_name": "Recent_Purchase_Months",
        "strict_boundaries": True,
    },
}

customer_tenure_years_constraint = {
    "constraint_class": "Positive",
    "constraint_parameters": {
        "column_name": "Customer_Tenure_Years",
        "strict_boundaries": True,
    },
}

policy_purchase_years_constraint = {
    "constraint_class": "Positive",
    "constraint_parameters": {
        "column_name": "Policy_Purchase_Years",
        "strict_boundaries": True,
    },
}
# For every row, the value in one column must be greater than a value in another.
policy_date_constraint = {
    "constraint_class": "Inequality",
    "constraint_parameters": {
        "low_column_name": "Policy_Issue_Date",
        "high_column_name": "Policy_Terminate_Date",
        "strict_boundaries": True,
    },
}

In [130]:
synthesizer.add_constraints(
    constraints=[
        policy_date_constraint
    ]  # recent_purchase_months_constraint, customer_tenure_years_constraint, policy_purchase_years_constraint,
)

### Proprocessing

In [131]:
synthesizer.auto_assign_transformers(data)
synthesizer.get_transformers()

{'CustomerID': AnonymizedFaker(function_name='bothify', function_kwargs={'text': '#########'}, cardinality_rule='unique'),
 'Age': FloatFormatter(learn_rounding_scheme=True, enforce_min_max_values=True),
 'Gender': UniformEncoder(),
 'Income': FloatFormatter(learn_rounding_scheme=True, enforce_min_max_values=True),
 'Marital_Status': UniformEncoder(),
 'Dependents': UniformEncoder(),
 'Claims': UniformEncoder(),
 'Policy': UniformEncoder(),
 'Premium_Amount': FloatFormatter(learn_rounding_scheme=True, enforce_min_max_values=True),
 'Insured_Amount': FloatFormatter(learn_rounding_scheme=True, enforce_min_max_values=True),
 'Payment_Timeliness': UniformEncoder(),
 'Payment_Frequency': UniformEncoder(),
 'Payment_Method': UniformEncoder(),
 'Recent_Payment_Months': FloatFormatter(learn_rounding_scheme=True, enforce_min_max_values=True),
 'Recent_Purchase_Months': FloatFormatter(learn_rounding_scheme=True, enforce_min_max_values=True),
 'Policy_Purchase_Years': FloatFormatter(learn_roundin

In [132]:
from rdt.transformers import AnonymizedFaker, PseudoAnonymizedFaker
from rdt.transformers import UniformEncoder


synthesizer.update_transformers(
    column_name_to_transformer={
        "CustomerID": AnonymizedFaker(
            function_name="bothify",
            function_kwargs={"text": "####"},
            cardinality_rule="match",
        ),
        # 'Gender': PseudoAnonymizedFaker(function_name='bothify', cardinality_rule='match'),
    }
)

In [133]:
synthesizer.get_transformers()

{'CustomerID': AnonymizedFaker(function_name='bothify', function_kwargs={'text': '####'}, cardinality_rule='match'),
 'Age': FloatFormatter(learn_rounding_scheme=True, enforce_min_max_values=True),
 'Gender': UniformEncoder(),
 'Income': FloatFormatter(learn_rounding_scheme=True, enforce_min_max_values=True),
 'Marital_Status': UniformEncoder(),
 'Dependents': UniformEncoder(),
 'Claims': UniformEncoder(),
 'Policy': UniformEncoder(),
 'Premium_Amount': FloatFormatter(learn_rounding_scheme=True, enforce_min_max_values=True),
 'Insured_Amount': FloatFormatter(learn_rounding_scheme=True, enforce_min_max_values=True),
 'Payment_Timeliness': UniformEncoder(),
 'Payment_Frequency': UniformEncoder(),
 'Payment_Method': UniformEncoder(),
 'Recent_Payment_Months': FloatFormatter(learn_rounding_scheme=True, enforce_min_max_values=True),
 'Recent_Purchase_Months': FloatFormatter(learn_rounding_scheme=True, enforce_min_max_values=True),
 'Policy_Purchase_Years': FloatFormatter(learn_rounding_sche

In [134]:
synthesizer.fit(data)

### Conditional Sampling
To synthesis selected customer with cross-sell

In [135]:
from sdv.sampling import Condition

# Setings Conditions
high_income = Condition(num_rows=250, column_values={"high_income": True})
under_insured = Condition(num_rows=250, column_values={"under_insured": True})
dependents_needs = Condition(num_rows=100, column_values={"dependents_needs": True})
recent_purchase = Condition(num_rows=100, column_values={"recent_purchase": True})

In [136]:
synthetic_high_income_data = synthesizer.sample_from_conditions(
    conditions=[high_income],
)
synthetic_under_insured_data = synthesizer.sample_from_conditions(
    conditions=[under_insured],
)
synthetic_dependents_needs_data = synthesizer.sample_from_conditions(
    conditions=[dependents_needs],
)
synthetic_recent_purchase_data = synthesizer.sample_from_conditions(
    conditions=[recent_purchase],
)
synthetic_general_data = synthesizer.sample(num_rows=100, batch_size=1_000)

Sampling rows: 100%|██████████| 100/100 [00:00<00:00, 736.14it/s]


In [137]:
# affix it to exisiting ID
under_insured_customer = data.loc[data["under_insured"] == True]
synthetic_under_insured_data["CustomerID"] = under_insured_customer[:300].reset_index()[
    "CustomerID"
]

high_income_customer = data.loc[data["high_income"] == True]
synthetic_high_income_data["CustomerID"] = high_income_customer[:250].reset_index()[
    "CustomerID"
]

dependents_customer = data.loc[data["dependents_needs"] == True]
synthetic_dependents_needs_data["CustomerID"] = dependents_customer[:100].reset_index()[
    "CustomerID"
]

recent_customer = data.loc[data["recent_purchase"] == True]
synthetic_recent_purchase_data["CustomerID"] = recent_customer[:100].reset_index()[
    "CustomerID"
]

# sampling
rand_zero_one_mask = data.sample(100).index
synthetic_general_data["CustomerID"] = data.loc[rand_zero_one_mask].reset_index()[
    "CustomerID"
]

In [138]:
combined_data = pd.concat(
    [
        data,
        synthetic_under_insured_data,
        synthetic_high_income_data,
        synthetic_dependents_needs_data,
        synthetic_recent_purchase_data,
        synthetic_general_data,
    ]
)

# Split to Static and Sequence (Updating) Table
- To ease clean up, and unifiying the data so that customers with same ID would have same ID.

In [139]:
static_data = combined_data[["CustomerID", "Gender", "Age"]]
static_data = static_data.groupby("CustomerID").last()
static_data["Cust_Birthday"] = [
    (datetime.datetime.now() - relativedelta(years=i)).date()
    for i in static_data["Age"]
]

In [140]:
static_data.head(5)

,Gender,Age,Cust_Birthday
CustomerID,,,
1,Male,37,1987-07-22
2,Female,46,1978-07-22
3,Female,43,1981-07-22
4,Female,59,1965-07-22
5,Male,56,1968-07-22


In [141]:
variable_data = combined_data[
    [
        "CustomerID",
        "Income",
        "Marital_Status",
        "Dependents",
        "Claims",
        "Policy",
        "Premium_Amount",
        "Insured_Amount",
        "Payment_Timeliness",
        "Payment_Frequency",
        "Payment_Method",
        "Recent_Payment_Months",
        "Recent_Purchase_Months",
        "Policy_Purchase_Years",
        "Smoker",
        "Policy_Issue_Date",
        "Policy_Terminate_Date",
        "under_insured",
        "high_income",
        "dependents_needs",
        "recent_purchase",
    ]
]

In [146]:
variable_data.head(5)

,CustomerID,Income,Marital_Status,Dependents,Claims,Policy,Premium_Amount,Insured_Amount,Payment_Timeliness,Payment_Frequency,...,Recent_Payment_Months,Recent_Purchase_Months,Policy_Purchase_Years,Smoker,Policy_Issue_Date,Policy_Terminate_Date,under_insured,high_income,dependents_needs,recent_purchase
0,1,11000,Divorced,0,None,Medical,2861,0,On-time,Monthly,...,9,28,1,False,1991-01-01,None,False,False,False,False
1,2,70000,Married,2,None,Medical,3148,0,On-time,Monthly,...,0,12,3,True,1996-05-01,None,True,False,False,False
2,3,44340,Married,2,Minor,Endowment,1455,14550,Late Payment,Annually,...,19,21,1,False,1995-03-03,2010-03-03,False,False,False,False
3,4,11000,Single,0,None,Endowment,1324,13240,On-time,Annually,...,13,11,15,False,1991-03-29,2005-03-29,False,False,False,False
4,5,11000,Married,2,None,Critical Illness,2857,100000,Late Payment,Annually,...,12,0,2,False,2014-04-01,2015-04-01,False,False,False,True


In [143]:
merged_data = static_data.merge(variable_data, on="CustomerID")

In [144]:
# Casting
merged_data["Policy_Terminate_Date"] = pd.to_datetime(
    merged_data["Policy_Terminate_Date"]
)
merged_data["Policy_Issue_Date"] = pd.to_datetime(merged_data["Policy_Issue_Date"])

merged_data["Policy_Issue_Date"] = [i.date() for i in merged_data["Policy_Issue_Date"]]
merged_data["Policy_Terminate_Date"] = [
    i.date() for i in merged_data["Policy_Terminate_Date"]
]

# Output

In [145]:
merged_data.to_parquet("data/L1/generated_data.parquet", index=False)